# Incoporation of heterologous genes in the iYO844 model for artemisinin production

_B. subtilis_ should be capable of producing farnesyl pyrophosphate (FPP), which is the end product of the MEP pathway. For production of artemisinin, six additional reactions are required. Additionally, a seventh reaction is added, as the enzymes incorporated also is responsible for generating a sideproduct. \

For each reaction a KEGG link is provided, as this is where the stoiciometric reactions were found.

In [9]:
# Loading the required packages
from cobra.io import read_sbml_model, write_sbml_model # To read the model and save new newly generated model
from cobra import Reaction, Metabolite # To creat new reactions and metabolites
from cameo.strain_design import pathway_prediction # To predict if there exist a pathway for a given metabolite

In [10]:
# Loading the the model
iYO844 = read_sbml_model("data/iYO844.xml")
iYO844_modified = iYO844.copy() # Generating a copy of iY0844 in a variable to add the reactions to the new model
iYO844

Name,iYO844
Memory address,0x07f9980a01910
Number of metabolites,990
Number of reactions,1250
Number of groups,0
Objective expression,1.0*BIOMASS_BS_10 - 1.0*BIOMASS_BS_10_reverse_8788b
Compartments,"cytosol, extracellular space"


First we make sure, that the MEP pathway is present in the current model iY40844 by searching for a pathway for creation of geranyl diphosphate, the second last step of the pathway.

In [11]:
# Might take a long time to run! 3-4 min
# Trying to predict if there already exists a patwhay for the end product, which there should
predictor = pathway_prediction.PathwayPredictor(iYO844) # Loading the model into the predictor tool
pathways = predictor.run(product='geranyl diphosphate', max_predictions=2) # Specifying metabolite. From KEGG pathway only two known pathways

,equation,lower_bound,upper_bound
MNXR69185,geranyl diphosphate + isopentenyl diphosphate ...,-1000.0,1000.0


,equation,lower_bound,upper_bound
MNXR55238,geranyl diphosphate + diphosphate <=> dimethyl...,-1000.0,1000.0


### First reaction
In the article "High level production of amorphadiene using Bacillus subtilis as an optimized terpenoid cell factory", a FPPS from  _Saccharomyces cerevisiae_ was inserted to convert geranyl pyrophosphate (GPP) to farnesyl pyrophosphate (FPP) as the last step in the MEP pathway instead of using the native IspA as FPPS. \

GPP --> FPP
Catalysed by FPPS \
Geranyl diphosphate + Isopentenyl diphosphate <=> Diphosphate + trans,trans-Farnesyl diphosphate \
https://www.genome.jp/entry/R02003

In [12]:
# Creating the reaction
reaction_1 = Reaction('FPPS') # FPPS: farnesyl pyrophosphate synthase

# Adding the metabolites based on the stoichiometric reaction
reaction_1.add_metabolites({iYO844.metabolites.get_by_id('grdp_c'): -1,
                            iYO844.metabolites.get_by_id('ipdp_c'): -1,
                            iYO844.metabolites.get_by_id('ppi_c'): 1,
                            iYO844.metabolites.get_by_id('frdp_c'): 1
                            })

print(reaction_1.build_reaction_string()) # Printing how the reaction looks
iYO844_modified.add_reactions([reaction_1]) # Adding the reaction to the new iY0844 model

grdp_c + ipdp_c --> frdp_c + ppi_c


### Second reaction
FPP --> amorphadiene / amorpha-4,11-diene + Diphosphate \
Catalysed by amorphadiene synthase (ADS) \
https://www.genome.jp/entry/R07630

In [13]:
# Creating the reaction
reaction_2 = Reaction('ADS') # ADS: amorphadiene synthase

# Defining the new metabolite, as it does not exists in GSM
amorphadiene = Metabolite(id='amorph_c', name='amorphadiene', compartment='c', formula='C15H24')

# Adding the metabolites based on the stoichiometric reaction
reaction_2.add_metabolites({iYO844.metabolites.get_by_id('frdp_c'): -1,
                            amorphadiene: 1,
                            iYO844.metabolites.get_by_id('ppi_c'): 1})
print(reaction_2.build_reaction_string()) # Printing how the reaction looks
iYO844_modified.add_reactions([reaction_2]) # Adding the reaction to the new iY0844 model

frdp_c --> amorph_c + ppi_c


### Third reaction
Amorpha-4,11-diene + Oxygen + [Reduced NADPH---hemoprotein reductase] <=> Artemisinic alcohol + [Oxidized NADPH---hemoprotein reductase] + H2O \
https://www.kegg.jp/entry/R10023

[Reduced NADPH---hemoprotein reductase] means: NADP+ + n reduced hemoprotein \
[Oxidized NADPH---hemoprotein reductase] means: NADPH + H+ + n oxidized hemoprotein \
https://en.wikipedia.org/wiki/NADPH—hemoprotein_reductase


In [14]:
# Creating the reaction
reaction_3 = Reaction('CYP71AV1') # CYP71AV1: amorphadiene oxidase

# Defining the new metabolite, as it does not exists in GSM
artemisinic_alcohol = Metabolite(id='artalc_c', name='artemisinic_alcohol', compartment='c', formula='C15H24O') 

# Adding the metabolites based on the stoichiometric reaction
reaction_3.add_metabolites({amorphadiene: -1,
                            iYO844.metabolites.get_by_id('o2_c'): -1,
                            iYO844.metabolites.get_by_id('nadp_c'): -1,
                            artemisinic_alcohol: 1, 
                            iYO844.metabolites.get_by_id('nadph_c'): 1,
                            iYO844.metabolites.get_by_id('h2o_c'): 1})

print(reaction_3.build_reaction_string()) # Printing how the reaction looks
iYO844_modified.add_reactions([reaction_3]) # Adding the reaction to the new iY0844 model

amorph_c + nadp_c + o2_c --> artalc_c + h2o_c + nadph_c


### Fourth reaction
Artemisinic alcohol + Oxygen + [Reduced NADPH---hemoprotein reductase] <=> Artemisinic aldehyde + [Oxidized NADPH---hemoprotein reductase] + 2 H2O \ https://www.genome.jp/entry/R10024


In [15]:
# Creating the reaction
reaction_4 = Reaction('ADH1_CYP71AV1') # ADH1: alcohol dehydrogenase, CYP71AV1: amorphadiene oxidase

# Defining the new metabolite, as it does not exists in GSM
artemisinic_aldehyde = Metabolite(id='artald_c', name='artemisinic aldehyde', compartment='c', formula='C15H22O')

# Adding the metabolites based on the stoichiometric reaction
reaction_4.add_metabolites({artemisinic_alcohol: -1,
                            iYO844.metabolites.get_by_id('o2_c'): -1,
                            iYO844.metabolites.get_by_id('nadp_c'): -1,
                            artemisinic_aldehyde: 1, 
                            iYO844.metabolites.get_by_id('nadph_c'): 1,
                            iYO844.metabolites.get_by_id('h2o_c'): 2})

print(reaction_4.build_reaction_string()) # Printing how the reaction looks
iYO844_modified.add_reactions([reaction_4]) # Adding the reaction to the new iY0844 model

artalc_c + nadp_c + o2_c --> artald_c + 2 h2o_c + nadph_c


### Fifth reaction
Artemisinic aldehyde + NADPH + H+  <=> (11R)-Dihydroartemisinic aldehyde + NADP+ \
https://www.genome.jp/entry/R10075

In [16]:
# Creating the reaction
reaction_5 = Reaction('DBR2') # DBR2: artemisinic aldehyde double-bond reductase

# Defining the new metabolite, as it does not exists in GSM
dihydroartemisinic_aldehyde = Metabolite(id='dihydroartald_c', name='dihydroartemisinic aldehyde', compartment='c', formula='C15H24O')

# Adding the metabolites based on the stoichiometric reaction
reaction_5.add_metabolites({artemisinic_aldehyde: -1,
                            iYO844.metabolites.get_by_id('nadph_c'): -1,
                            iYO844.metabolites.get_by_id('h_c'): -1,
                            dihydroartemisinic_aldehyde: 1, 
                            iYO844.metabolites.get_by_id('nadp_c'): 1})

print(reaction_5.build_reaction_string()) # Printing how the reaction looks
iYO844_modified.add_reactions([reaction_5]) # Adding the reaction to the new iY0844 model

artald_c + h_c + nadph_c --> dihydroartald_c + nadp_c


### Sixth reaction
Dihydroartemisinic aldehyde + Oxygen + [Reduced NADPH---hemoprotein reductase] <=> Dihydroartemisinic acid + [Oxidized NADPH---hemoprotein reductase] + H2O

Modified from: https://www.genome.jp/entry/R10025


In [17]:
# Creating the reaction
reaction_6 = Reaction('ALDH1_CYP71AV1') # ALDH1: aldehyde dehydrogenase 1, CYP71AV1: amorphadiene oxidase

# Defining the new metabolite, as it does not exists in GSM
dihydroartemisinic_acid = Metabolite(id='dihydroartaci_c', name='dihydroartemisinic acid', compartment='c', formula='C15H24O2')

# Adding the metabolites based on the stoichiometric reaction
reaction_6.add_metabolites({artemisinic_alcohol: -1,
                            iYO844.metabolites.get_by_id('o2_c'): -1,
                            iYO844.metabolites.get_by_id('nadp_c'): -1,
                            dihydroartemisinic_acid: 1, 
                            iYO844.metabolites.get_by_id('nadph_c'): 1,
                            iYO844.metabolites.get_by_id('h2o_c'): 1})

print(reaction_6.build_reaction_string()) # Printing how the reaction looks
iYO844_modified.add_reactions([reaction_6]) # Adding the reaction to the new iY0844 model

artalc_c + nadp_c + o2_c --> dihydroartaci_c + h2o_c + nadph_c


### Seventh reaction
As the same enzymes responsible for reaction six also converts artemisinic aldehyde into artemisinic acid, this reaction also needs to be added to _B. subtilis_.

Artemisinic aldehyde + Oxygen + [Reduced NADPH---hemoprotein reductase] <=> Artemisinate + [Oxidized NADPH---hemoprotein reductase] + H2O \
https://www.genome.jp/entry/R10025 \

[Reduced NADPH---hemoprotein reductase] means: NADP+ + n reduced hemoprotein \
[Oxidized NADPH---hemoprotein reductase] means: NADPH + H+ + n oxidized hemoprotein

In [18]:
# Creating the reaction
reaction_7 = Reaction('ALDH1_CYP71AV1_2') # ALDH1: aldehyde dehydrogenase 1, CYP71AV1: amorphadiene oxidase

# Defining the new metabolite, as it does not exists in GSM
artemisinic_acid = Metabolite(id='artaci_c', name='artemisinic acid', compartment='c', formula='C15H22O2')

# Adding the metabolites based on the stoichiometric reaction
reaction_7.add_metabolites({artemisinic_alcohol: -1,
                            iYO844.metabolites.get_by_id('o2_c'): -1,
                            iYO844.metabolites.get_by_id('nadp_c'): -1,
                            artemisinic_acid: 1, 
                            iYO844.metabolites.get_by_id('nadph_c'): 1,
                            iYO844.metabolites.get_by_id('h2o_c'): 1})

print(reaction_7.build_reaction_string()) # Printing how the reaction looks
iYO844_modified.add_reactions([reaction_7]) # Adding the reaction to the new iY0844 model

artalc_c + nadp_c + o2_c --> artaci_c + h2o_c + nadph_c


### Testing implementation
Testing to see if dihydroartemisinic aldehyde can be produced after implementing the seven reactions to the GSM \
OBS! Maybe we should test for all the reactions?

In [19]:
# Specifying which metabolite, we want to test the production of
dihydroartaci = iYO844_modified.add_boundary(iYO844_modified.metabolites.get_by_id('dihydroartaci_c'), type='demand')

# OBS! Comments needs to be added to this 
with iYO844_modified :
    iYO844_modified.objective = dihydroartaci
    sol = iYO844_modified.optimize()
    print(sol.objective_value)

0.36342857142857143


From this it can be seen, that here is a production of dihydroartemisinic aldehyde within the cell.

### Saving the model
The current model is saved as "iYO844_modified.xml" in the data folder.

In [20]:
# Saving the variable as a new xml file
write_sbml_model(iYO844_modified, 'data/iYO844_modified.xml')

In [28]:
# iYO844_modified.medium
iYO844_modified.metabolites.dihydroartaci_c

Metabolite identifier,dihydroartaci_c
Name,dihydroartemisinic acid
Memory address,0x07f9964c0bc10
Formula,C15H24O2
Compartment,c
In 2 reaction(s),"ALDH1_CYP71AV1, DM_dihydroartaci_c"


# FOR MEDIUM ANALYSIS
OBS! Move to another file

In [34]:
iYO844_modified.metabolites.get_by_id('glc__D_c')

Metabolite identifier,glc__D_c
Name,D-Glucose
Memory address,0x07f996711cb50
Formula,C6H12O6
Compartment,c
In 20 reaction(s),"LACZ, MPL, MAL6PG, BGLA1, MLTG4, MLTG2, HEX1, MALT, MLTG5, MLTG1, TRE6PH, MLTG3, SUCR, GALS3, XYLI2, GLCt2, MLTG6, G1PPDM, MLTG7, G1Dx"


In [36]:
iYO844_modified.objective = iYO844_modified.reactions.ALDH1_CYP71AV1

dihydroartaci_production = iYO844_modified.optimize().objective_value
print('Maximum productivity =', dihydroartaci_production, 'mmol/gDW*h')

#maximum_yield = dihydroartaci_production / (-1*(iYO844_modified.reactions.BOUNDARY_glc__D_c.flux))
#print('Maximum theoretical yield =', maximum_yield, 'mmol-dihydroaci/mmol-glc')


Maximum productivity = 0.36342857142857177 mmol/gDW*h
